In [257]:
import pandas as pd

In [272]:
smpl_corr_df = pd.read_csv('/data/archive/compendium/v7/v7_all-by-all_2018-05-14.tsv', sep='\t', index_col=0)
node_coord_df = pd.read_csv('/data/scratch/hb_deleteMe/hexmapCoordsCompV7.tab', sep='\t', header=None, index_col=0)

In [273]:
node_coord_df.head()

,1,2
0,,
TCGA-2G-AAHL-01,60,103
TCGA-VF-A8AE-01,56,100
TCGA-FU-A23L-01,284,204
TCGA-BP-4352-01,309,62
TCGA-D8-A1JL-01,178,173


In [260]:
smpl_corr_df.head()

,THR15_0330_S01,THR29_0776_S01,THR11_0247_S01,THR08_0162_S01,THR22_0588_S01,TARGET-40-0A4I65-01A-01R,THR09_0563_S01,THR30_0827_S01,THR19_0459_S01,THR30_0851_S01,...,TCGA-18-3414-01,TCGA-13-1481-01,TCGA-BP-4803-01,TCGA-A8-A09K-01,TCGA-61-1907-01,TCGA-IB-7885-01,TCGA-95-7947-01,TCGA-VQ-AA6F-01,TCGA-BR-8588-01,TCGA-DD-A115-01
sample,,,,,,,,,,,,,,,,,,,,,
THR15_0330_S01,1.000000,0.827806,0.793105,0.713853,0.815943,0.866218,0.814270,0.793288,0.801691,0.804125,...,0.820701,0.809572,0.821837,0.785381,0.793899,0.837263,0.806405,0.778061,0.806284,0.773952
THR29_0776_S01,0.827806,1.000000,0.813258,0.717080,0.806533,0.810998,0.824333,0.819525,0.796094,0.818346,...,0.781826,0.811453,0.780088,0.781035,0.773222,0.786526,0.775950,0.755538,0.774534,0.732119
THR11_0247_S01,0.793105,0.813258,1.000000,0.744485,0.816026,0.784852,0.776780,0.773365,0.897544,0.793490,...,0.759506,0.776582,0.801677,0.778746,0.765350,0.791679,0.786850,0.763623,0.780823,0.731706
THR08_0162_S01,0.713853,0.717080,0.744485,1.000000,0.801457,0.717347,0.803952,0.743840,0.691500,0.739948,...,0.691215,0.685508,0.700407,0.749525,0.716561,0.680700,0.738045,0.724004,0.716712,0.660873
THR22_0588_S01,0.815943,0.806533,0.816026,0.801457,1.000000,0.784031,0.873419,0.774972,0.779460,0.789145,...,0.783592,0.779459,0.811841,0.815244,0.799217,0.814045,0.820879,0.811291,0.815159,0.743425


In [276]:
%%time
coord_to_sampleDict = {}



for sample in node_coord_df.index:
    '''Reverse the key:value association from the hexagonal coordinate file to be 
    coordinates:sample_ID from sample_ID:coordinates'''
    x, y = node_coord_df.loc[sample]
    coord_to_sampleDict[(x, y)] = sample
    
def get_nbrs_odd(x, y):
    '''For hexagonal coordinates, surrounding hexagon coordinates are different for nodes with odd X-values 
    than nodes with even X-values.Enter X and Y coords for a node with an odd X-value and if it has neighbors, 
    they will be returned in a list from this method. Neighbors for nodes with odd X-coordinates have neighbors at:
    (x, y-1)
    (x+1, y)
    (x-1, y-1)
    (x, y+1)
    (x+1, y-1)
    (x-1, y)'''
    
    if (x, y-1) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x, y-1)]
    if (x+1, y) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x+1, y)]
    if (x-1, y-1) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x-1, y-1)]
    if (x, y+1) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x, y+1)]
    if (x+1, y-1) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x+1, y-1)]
    if (x-1, y) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x-1, y)]
        
def get_nbrs_even(x, y):
    '''For hexagonal coordinates, surrounding hexagon coordinates are different for nodes with even X-values
    than nodes with odd X-values. Enter X and Y coords for a node with an even X-value and if it has neighbors, 
    they will be returned in a list from this method. Neighbors for nodes with even X-coordinates have neighbors at:
    (x, y-1)
    (x+1, y)
    (x+1, y+1)
    (x, y+1)
    (x-1, y+1)
    (x+1, y)'''
    
    if (x, y-1) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x, y-1)]
    if (x+1, y) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x+1, y)]
    if (x+1, y+1) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x+1, y+1)]
    if (x, y+1) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x, y+1)]
    if (x-1, y+1) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x-1, y+1)]
    if (x+1, y) in coord_to_sampleDict.keys():
        yield coord_to_sampleDict[(x+1, y)]
    

CPU times: user 2.56 s, sys: 0 ns, total: 2.56 s
Wall time: 2.56 s


In [277]:
%%time
avg_nbr_corr = pd.Series()
lowest_corr = 1.0
soI='None'
low_cor_nbr='None'

for coords in coord_to_sampleDict.keys():
    if coords[0] % 2 != 0:
        '''Check if X-coord is even. If it is, add all of its neighbors to a list and 
        calculate the average correlation value of the other samples of that list. Will 
        then assign that average correlation value to the sample.'''
        neighbors = get_nbrs_odd(coords[0], coords[1])
        addCorr = 0
        length = 0
        for nbrs in neighbors:
            addCorr += smpl_corr_df.loc[coord_to_sampleDict[(coords[0], coords[1])]][nbrs]
            length += 1
        if length == 0:
            avgCorr = 0
        if length > 0:
            avgCorr = addCorr/length
        avg_nbr_corr[coord_to_sampleDict[(coords[0], coords[1])]] = avgCorr
        
        
    else:
        '''If X-coord is not even, then it is odd. For each sample with odd X-coordinates, 
        will add all of its neighbors to a list and calculate the average correlation value 
        of the other samples of that list. Will then assign that average correlation value to the sample.'''
        neighbors = get_nbrs_even(coords[0], coords[1])
        addCorr = 0
        length = 0
        for nbrs in neighbors:
            addCorr += smpl_corr_df.loc[coord_to_sampleDict[(coords[0], coords[1])]][nbrs]
            length += 1
        if length == 0:
            avgCorr = 0
        if length > 0:
            avgCorr = addCorr/length
        avg_nbr_corr[coord_to_sampleDict[(coords[0], coords[1])]] = avgCorr
        

CPU times: user 37.8 s, sys: 0 ns, total: 37.8 s
Wall time: 37.8 s


In [278]:
#Print average neighbor correlation for each sample to a tsv file
#Still having difficulties dropping the '0' index after the AverageCorrelation index
avg_nbr_corr.to_csv(path='Average_Neighbor_Correlation.tsv', sep='\t', header=True, index_label=('sample ID','AverageCorrelation'))


In [ ]:
# '''This kernel is used to find the sample that has lowest correlation with its touching neighbors.'''

avg_nbr_corr.head()
lowest_corr = 1.0

for values in avg_nbr_corr.values:
    if values < lowest_corr and values > 0:
        lowest_corr = values
for samps in avg_nbr_corr.index:
    if avg_nbr_corr.loc[samps] == lowest_corr:
        name = samps
for nodes in node_coord_df.index:
    if node_coord_df.loc[nodes][0] == name:
        coordXY = (node_coord_df.loc[nodes][1],node_coord_df.loc[nodes][2])

        
print(lowest_corr)
print(name)
print(coordXY)